In [7]:
import pandas as pd
import numpy as np
import joblib
import json
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings("ignore")
import os


# STEP 1: LOAD DATA & FEATURE ENGINEERING
df = pd.read_csv("F:/projects/personal project/weather prediction_v1/data/weather_atmospheric_processed.csv", parse_dates=['time'], index_col='time')

# Define model directory path
model_dir = "F:/projects/personal project/weather prediction_v1/models/"

# Create lag features
for col in df.columns:
    df[f"{col}_lag1"] = df[col].shift(1)

# Rolling means
for col in df.columns:
    df[f"{col}_roll3"] = df[col].rolling(window=3).mean()

# Time-based features
df['month'] = df.index.month
df['dayofyear'] = df.index.dayofyear

# Drop rows with NaNs due to shifting/rolling
df.dropna(inplace=True)

# STEP 2: MULTI-STEP TARGET CREATION (t+1 to t+3)
target_var = 'TMP'  # Target to forecast
df['TMP_t+1'] = df[target_var].shift(-1)
df['TMP_t+2'] = df[target_var].shift(-2)
df['TMP_t+3'] = df[target_var].shift(-3)
df.dropna(inplace=True)

# STEP 3: TRAIN-TEST SPLIT & SCALING
features = df.drop(columns=['TMP_t+1', 'TMP_t+2', 'TMP_t+3'])
targets = df[['TMP_t+1', 'TMP_t+2', 'TMP_t+3']]

# Save feature names
feature_names = features.columns.tolist()
with open("feature_names.json", "w") as f:
    json.dump(feature_names, f)
print(f"Feature names saved to feature_names.json: {feature_names}")

# Make predictions for the next 3 days
print("\n### Making predictions for the next 3 days in Kelvin ###")

# Load the trained models and scaler
try:
    scaler = joblib.load(os.path.join(model_dir, "scaler.pkl"))
    rf_model_t1 = joblib.load(os.path.join(model_dir, "rf_model_TMP_t+1.pkl"))
    rf_model_t2 = joblib.load(os.path.join(model_dir, "rf_model_TMP_t+2.pkl"))
    rf_model_t3 = joblib.load(os.path.join(model_dir, "rf_model_TMP_t+3.pkl"))
    lstm_model_t1 = load_model(os.path.join(model_dir, "lstm_model_TMP_t+1.h5"), compile=False)
    lstm_model_t2 = load_model(os.path.join(model_dir, "lstm_model_TMP_t+2.h5"), compile=False)
    lstm_model_t3 = load_model(os.path.join(model_dir, "lstm_model_TMP_t+3.h5"), compile=False)

    # Get the last row of the original dataframe for prediction
    last_row_features = features.iloc[-1].values.reshape(1, -1)

    # Scale the last row using the loaded scaler
    scaled_last_row_features = scaler.transform(last_row_features)

    # Reshape for LSTM (samples, timesteps=1, features)
    scaled_last_row_features_lstm = scaled_last_row_features.reshape((scaled_last_row_features.shape[0], 1, scaled_last_row_features.shape[1]))

    # Make predictions
    rf_pred_t1 = rf_model_t1.predict(scaled_last_row_features)[0]
    rf_pred_t2 = rf_model_t2.predict(scaled_last_row_features)[0]
    rf_pred_t3 = rf_model_t3.predict(scaled_last_row_features)[0]

    lstm_pred_t1 = lstm_model_t1.predict(scaled_last_row_features_lstm)[0][0]
    lstm_pred_t2 = lstm_model_t2.predict(scaled_last_row_features_lstm)[0][0]
    lstm_pred_t3 = lstm_model_t3.predict(scaled_last_row_features_lstm)[0][0]

    print("\n--- Random Forest Predictions ---")
    print(f"Predicted Temperature for t+1: {rf_pred_t1:.2f}K")
    print(f"Predicted Temperature for t+2: {rf_pred_t2:.2f}K")
    print(f"Predicted Temperature for t+3: {rf_pred_t3:.2f}K")

    print("\n--- LSTM Predictions ---")
    print(f"Predicted Temperature for t+1: {lstm_pred_t1:.2f}K")
    print(f"Predicted Temperature for t+2: {lstm_pred_t2:.2f}K")
    print(f"Predicted Temperature for t+3: {lstm_pred_t3:.2f}K")

except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure all required model and data files are in the directory.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Feature names saved to feature_names.json: ['plevel', 'HGT', 'RH', 'TMP', 'UGRD', 'VGRD', 'plevel_lag1', 'HGT_lag1', 'RH_lag1', 'TMP_lag1', 'UGRD_lag1', 'VGRD_lag1', 'plevel_roll3', 'HGT_roll3', 'RH_roll3', 'TMP_roll3', 'UGRD_roll3', 'VGRD_roll3', 'plevel_lag1_roll3', 'HGT_lag1_roll3', 'RH_lag1_roll3', 'TMP_lag1_roll3', 'UGRD_lag1_roll3', 'VGRD_lag1_roll3', 'month', 'dayofyear']

### Making predictions for the next 3 days in Kelvin ###
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step

--- Random Forest Predictions ---
Predicted Temperature for t+1: 285.88K
Predicted Temperature for t+2: 290.25K
Predicted Temperature for t+3: 295.03K

--- LSTM Predictions ---
Predicted Temperature for t+1: 287.24K
Predicted Temperature for t+2: 290.86K
Predicted Temperature for t+3: 295.21K
